# **Intro to common CNN APIs**
此份程式碼會介紹在 CNN model 當中常使用的 Layers。

## 本章節大綱
* [Conv2D( filters, kernel_size, strides, use_bias)](#Conv2D)
  * [use_bias](#use-bias)
  * [Multi-Channels](#Multi-Channels-with-1-Filter)
  * [filters](#filters)
  * [kernel_size](#kernel-_-size)
  * [strides](#strides)
* [Flatten](#Flatten)
* [Padding](#Padding)
* [Pooling](#Pooling)

In [ ]:
# 下載課程所需檔案
!wget -q "https://github.com/TA-aiacademy/course_3.0/releases/download/CVCNN_Data/cnn_part2_data.zip"
!unzip -q cnn_part2_data.zip

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D

In [ ]:
input_img = np.array([[0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 1, 1, 0],
                      [0, 1, 1, 1, 1, 0],
                      [0, 0, 1, 0, 1, 0],
                      [0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0]], dtype='float32')

In [ ]:
input_img.shape

In [ ]:
input_img = input_img[np.newaxis, ..., np.newaxis]
print(input_img.shape)
print("(batch_size, height, width, channel)")

* ## Conv2D
![conv2D](https://hackmd.io/_uploads/Hy6RbRUIp.gif)

In [ ]:
def kernel_init(shape, dtype=None):
    filter_init = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]], dtype='float32')
    # height, width, channel, filters
    filter_init = filter_init.reshape((3, 3, 1, 1))
    return tf.Variable(filter_init)

In [ ]:
conv_result = Conv2D(filters=1, kernel_size=(3, 3), strides=(1, 1),
                     kernel_initializer=kernel_init)(input_img)

In [ ]:
conv_result = conv_result.numpy()

In [ ]:
print(conv_result.shape)
print(conv_result.squeeze())

[(back...)](#Convolution2D)

* ## use bias
![use bias](https://hackmd.io/_uploads/BkFRfR8La.gif)

In [ ]:
bias_result = Conv2D(filters=1, kernel_size=(3, 3), strides=(1, 1),
                     kernel_initializer=kernel_init,
                     use_bias=True,
                     bias_initializer='ones')(input_img)

bias_result = bias_result.numpy()

In [ ]:
print(bias_result.shape)
print(bias_result.squeeze())

[(back...)](#Convolution2D)

* ## Multi Channels with 1 Filter
![Multi Channels with 1 filter](https://hackmd.io/_uploads/S1q1m08I6.gif)
![QEjI0jq](https://hackmd.io/_uploads/By9e70I8a.png)


In [ ]:
input_img = np.load("./data/conv2d_multichannel_input.npy")
print(input_img.shape)
print(input_img.dtype)

In [ ]:
input_img = input_img[np.newaxis, ...]
print(input_img.shape)
print("(Batch_size, Height, Width, Channel)")

In [ ]:
input_img = input_img.astype("float32")
print(input_img.dtype)

In [ ]:
filter_init = np.load("./data/conv2d_multichannelfilter.npy")
print(filter_init.shape)
print("(Height, Width, Channel, Num of Filters)")

In [ ]:
kernel_init = tf.constant_initializer(filter_init)

In [ ]:
multichannel = Conv2D(filters=1, kernel_size=(3, 3), strides=(1, 1),
                      kernel_initializer=kernel_init)(input_img)

multichannel = multichannel.numpy()

In [ ]:
print(multichannel.shape)
print(multichannel.squeeze())

[(back...)](#Convolution2D)

* ## filters
![filters](https://hackmd.io/_uploads/BJV77RUU6.gif)

In [ ]:
multi_filter_init = np.zeros((3, 3, 3, 8))
for i in range(8):
    multi_filter_init[:, :, :, i] = filter_init.squeeze()
multi_filter_init = multi_filter_init.astype('float32')

print(multi_filter_init.shape)

In [ ]:
kernel_init = tf.constant_initializer(multi_filter_init)

In [ ]:
multifilter = Conv2D(8, (3, 3), strides=(1, 1),
                     kernel_initializer=kernel_init)(input_img)

multifilter = multifilter.numpy()

In [ ]:
print(multifilter.shape)
print(multifilter.squeeze())

[(back...)](#Convolution2D)

* ## strides
![strides](https://hackmd.io/_uploads/r17N708Up.gif)

In [ ]:
input_img = np.load("./data/conv2d_1channel_input.npy")

In [ ]:
filter_init = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]], dtype='float32')
filter_init = filter_init.reshape((3, 3, 1, 1))
kernel_init = tf.constant_initializer(filter_init)

In [ ]:
stride_result = Conv2D(1, (3, 3), strides=(2, 2),
                       kernel_initializer=kernel_init)(input_img)

stride_result = stride_result.numpy()

In [ ]:
print(stride_result.shape)
print(stride_result.squeeze())

![image](https://hackmd.io/_uploads/Sk__X0LIa.png)

# Flatten

* [Way1-Reshape](#Way1---Reshape)
* [Way2-Flatten](#Way2---Flatten)

In [ ]:
import numpy as np
from tensorflow.keras.layers import Flatten, Reshape

In [ ]:
input_img1 = np.array([[0, 1, 2, 3],
                       [4, 5, 6, 7],
                       [8, 9, 10, 11],
                       [12, 13, 14, 15]], dtype='float32')
input_img1 = input_img1[np.newaxis, ..., np.newaxis]

* ## Way1 - Reshape

In [ ]:
reshape_result = Reshape(target_shape=(-1,))(input_img1)
reshape_result = reshape_result.numpy()

In [ ]:
print(input_img1.shape)
print(reshape_result.shape)
print(reshape_result)

In [ ]:
input_img2 = input_img1.copy()
for _ in range(3):
    input_img2 = np.concatenate([input_img2, input_img2], -1)
print(input_img2.shape)

In [ ]:
reshape_result = Reshape(target_shape=(-1,))(input_img2)
reshape_result = reshape_result.numpy()
print(reshape_result.shape)
print(reshape_result)

[(back...)](#Flatten)

* ## Way2 - Flatten

![Flatten](https://hackmd.io/_uploads/ByBFmR8Ia.gif)

In [ ]:
flatten_result = Flatten()(input_img1)
flatten_result = flatten_result.numpy()

In [ ]:
print(input_img1.shape)
print(flatten_result.shape)
print(flatten_result)

![Flatten_M](https://hackmd.io/_uploads/HyZ5Q08Ia.gif)

In [ ]:
flatten_result = Flatten()(input_img2)
flatten_result = flatten_result.numpy()

In [ ]:
print(flatten_result.shape)
print(flatten_result)

[(back...)](#Flatten)

# Padding

* [padding='VALID'](#padding='VALID')
* [padding='SAME'](#padding='SAME')
* [ZeroPadding](#ZeroPadding)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, ZeroPadding2D

In [ ]:
input_img = np.array([[0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 1, 1, 0],
                      [0, 1, 1, 1, 1, 0],
                      [0, 0, 1, 0, 1, 0],
                      [0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0]], dtype='float32')
input_img = input_img[np.newaxis, ..., np.newaxis]

In [ ]:
def kernel_init(shape, dtype=None):
    filter_init = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    filter_init = filter_init.reshape((3, 3, 1, 1))
    return tf.Variable(filter_init, dtype=tf.float32)

* ## padding='VALID'

In [ ]:
nopad_result = Conv2D(1, (3, 3), padding='VALID',
                      kernel_initializer=kernel_init)(input_img)

nopad_result = nopad_result.numpy()

In [ ]:
print(input_img.shape)
print(nopad_result.shape)
print(nopad_result.squeeze())

[(back...)](#Padding)

* ## padding='SAME'
![padding_s](https://hackmd.io/_uploads/HyQBZ1PUp.gif)

In [ ]:
pad_result = Conv2D(1, (3, 3), padding='SAME',
                    kernel_initializer=kernel_init)(input_img)

pad_result = pad_result.numpy()

In [ ]:
print(input_img.shape)
print(pad_result.shape)
print(pad_result.squeeze())

[(back...)](#Padding)

## ZeroPadding

In [ ]:
zero_padding = ZeroPadding2D(padding=(1, 1))(input_img)
zero_result = Conv2D(1, (3, 3),
                     kernel_initializer=kernel_init)(zero_padding)

zero_padding = zero_padding.numpy()
zero_result = zero_result.numpy()

In [ ]:
print(input_img.shape)
print(zero_padding.shape)
print(zero_padding.squeeze())

In [ ]:
print(zero_result.shape)
print(zero_result.squeeze())

[(back...)](#Padding)

# Pooling


* [Average Pooling](#Average-Pooling)
* [Max Pooling](#Max-Pooling)

![image](https://hackmd.io/_uploads/H1LFb1D8p.png)

In [ ]:
import numpy as np
from tensorflow.keras.layers import AveragePooling2D, MaxPool2D

In [ ]:
input_img = np.array([[1, 2, 2, 0],
                      [1, 2, 3, 2],
                      [3, 1, 3, 2],
                      [0, 2, 0, 2]], dtype='float32').reshape((1, 4, 4, 1))

* ## Average Pooling

![avg pool](https://hackmd.io/_uploads/HkgoW1v86.gif)


In [ ]:
avg_result = AveragePooling2D()(input_img)
avg_result = avg_result.numpy()

In [ ]:
print(input_img.shape)
print(avg_result.shape)
print(avg_result.squeeze())

[(back...)](#Pooling)

* ## Max Pooling

![max pool](https://hackmd.io/_uploads/rkCob1P8p.gif)

In [ ]:
max_result = MaxPool2D()(input_img)
max_result = max_result.numpy()

In [ ]:
print(input_img.shape)
print(max_result.shape)
print(max_result.squeeze())

[(back...)](#Pooling)

# GlobalPooling

* [Global Average Pooling](#Global-Average-Pooling)
* [Global Max Pooling](#Global-Max-Pooling)

In [ ]:
import numpy as np
from tensorflow.keras.layers import (GlobalAveragePooling2D,
                                     GlobalMaxPooling2D)

In [ ]:
input_img = np.load("./data/globalpooling_input.npy")[np.newaxis, ...]
input_img = input_img.astype('float32')

* ## Global Average Pooling

![GAP](https://hackmd.io/_uploads/Bk9n-1PIp.gif)


In [ ]:
print(input_img.shape)
print(input_img[..., 0])

In [ ]:
avg_result = GlobalAveragePooling2D()(input_img)
avg_result = avg_result.numpy()

In [ ]:
print(avg_result.shape)
print(avg_result.squeeze())

In [ ]:
input_img.mean((1, 2))

[(back...)](#GlobalPooling)

* ## Global Max Pooling

![GMP](https://hackmd.io/_uploads/B1mJz1DL6.gif)


In [ ]:
max_result = GlobalMaxPooling2D()(input_img)
max_result = max_result.numpy()

In [ ]:
print(input_img.shape)
print(input_img[..., 0])

In [ ]:
print(max_result.shape)
print(max_result.squeeze())

[(back...)](#GlobalPooling)